In [28]:
from paddleocr import PaddleOCR, draw_ocr
from easyocr import Reader
import pytesseract
from pytesseract import Output
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image


In [29]:
paddlereader = PaddleOCR(lang='en')
easyreader = Reader(['en'])
pytesseract.pytesseract.tesseract_cmd = r'D:\Program Files\Tesseract-OCR\tesseract.exe'

[2022/05/23 06:17:47] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\grace/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\grace/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_pa

In [30]:
img_dir = '../receipts/pictures'
processed_dir = '../receipts/processed'
result_dir = '../receipts/results'
images = [os.path.splitext(filename)[0] for filename in os.listdir(img_dir)]
preprocessed = [os.path.splitext(filename)[0] for filename in os.listdir(processed_dir)]

In [ ]:
for image in images:
    img_path = os.path.join(processed_dir, image + '.jpg')
    result_path = os.path.join(processed_dir, image + '_easyocr.jpg')
    text_path = os.path.join(processed_dir, image + '_easyocr.txt')

    result = easyreader.readtext(img_path)

    img = cv2.imread(img_path)
    spacer = 100
    font = cv2.FONT_HERSHEY_SIMPLEX

    for detection in result: 
        top_left = tuple(detection[0][0])
        bottom_right = tuple(detection[0][2])
        text = detection[1]
        try:
            img = cv2.rectangle(img,top_left,bottom_right,(0,255,0),3)
            spacer+=15
        except:
            pass

    with open(text_path,'w') as f:
        f.write("\n".join(str(item) for item in result))

    plt.figure(figsize=(10,10))
    plt.imshow(img)
    plt.savefig(result_path)

In [31]:
for image in images:
    img_path = os.path.join(processed_dir, image + '.jpg')
    result_path = os.path.join(processed_dir, image + '_paddle.jpg')
    text_path = os.path.join(processed_dir, image + '_paddle.txt')

    result = paddlereader.ocr(img_path, cls=False)

    with open(text_path,'w') as f:
        f.write("\n".join(str(item) for item in result))

    image = Image.open(img_path).convert('RGB')

    boxes = [line[0] for line in result]
    txts = [line[1][0] for line in result]
    scores = [line[1][1] for line in result]
    im_show = draw_ocr(image, boxes)

    im_show = Image.fromarray(im_show)
    im_show.save(result_path)

[2022/05/23 06:18:02] ppocr DEBUG: dt_boxes num : 56, elapse : 0.18355035781860352
[2022/05/23 06:18:02] ppocr DEBUG: rec_res num  : 56, elapse : 0.32607412338256836
[2022/05/23 06:18:02] ppocr DEBUG: dt_boxes num : 20, elapse : 0.028759002685546875
[2022/05/23 06:18:02] ppocr DEBUG: rec_res num  : 20, elapse : 0.03809046745300293
[2022/05/23 06:18:03] ppocr DEBUG: dt_boxes num : 27, elapse : 0.02800607681274414
[2022/05/23 06:18:03] ppocr DEBUG: rec_res num  : 27, elapse : 0.05117607116699219
[2022/05/23 06:18:03] ppocr DEBUG: dt_boxes num : 27, elapse : 0.03000640869140625
[2022/05/23 06:18:03] ppocr DEBUG: rec_res num  : 27, elapse : 0.049010515213012695
[2022/05/23 06:18:03] ppocr DEBUG: dt_boxes num : 26, elapse : 0.030006885528564453
[2022/05/23 06:18:03] ppocr DEBUG: rec_res num  : 26, elapse : 0.04801130294799805


In [27]:
for image in images:
    img_path = os.path.join(processed_dir, image + '.jpg')
    result_path = os.path.join(processed_dir, image + '_tesseract.jpg')
    text_path = os.path.join(processed_dir, image + '_tesseract.txt')

    img = cv2.imread(img_path)
    h, w, _ = img.shape

    d = pytesseract.image_to_data(img, output_type=Output.DATAFRAME)
    boxes = []

    d = d[d.conf != -1]
    d = d[d.text != ' ']
    d = d[d.text != '  ']

    d.to_csv(text_path, header=0, index=0)

    for i, row in d.iterrows():
        x1 = row['left']
        y1 = row['top']
        x2 = row['left']+row['width']
        y2 = y1
        x3 = x2
        y3 = row['top']+row['height']
        x4 = x1
        y4 = y3

        boxes.append([x1,y1,x2,y2,x3,y3,x4,y4,row['text']])
        (x, y, w, h) = (row['left'], row['top'], row['width'], row['height'])
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imwrite(result_path, img)
